# Visualisierung der Kompetenzen-Excel-Tabelle

In [ ]:
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import pandas as pd
import re

load_dotenv()
data = os.getenv('data')
sciebo = data[:-4]
file = sciebo + r"\DH-Ruhr\Recherche\Kompetenzen\DH_Kompetenzen_Übersicht_V2.2.xlsx"


In [ ]:
df = pd.read_excel(file, index_col="id")

df.head()

In [ ]:
def brrrrrr(string, cutoff=20):
    brstring = ""
    while len(string) > cutoff:
        space_index = string[cutoff:].find(" ")
        if space_index == -1:
            break
        brstring += string[:cutoff + space_index].strip() + "<br>"
        string = string[cutoff + space_index:].strip()
    
    brstring += string
    return brstring
    
        

teststr = df['Atomic Skills'].iloc[92]
brrrrrr(teststr,20)

In [ ]:

df["AS"] = df["Atomic Skills"].apply(lambda x: brrrrrr(x,15))
df["Prio_"] = df["Prio"].apply(lambda x: 4-x)

In [ ]:
df.keys()

In [ ]:
# df[['Atomic Skills', 'Bereich', 'Unterbereich', 'Prio', 'Zeitaufwand','Lernzieltyp', 'Modul']].to_csv("dh_cert_skills_RUB.csv")

In [ ]:
RUB_palette = {'Blau': '#17365c',
               'Grün': '#8dae10',
               'Grau': '#e7e7e7',
               'Gelb': '#ffcc00',
               'Orange': '#ed7102',
               'Rot': '#e6332a',
               'Dunkelrot': '#b61e3e',
               'Beige': '#c1baa3',
               'Dunkelgrün': '#8c8751',
               'Braun': '#9c5516',
               'Dunkelbraun': '#59201b'}


bereich_colors = {'DH Grundlagen': 'Grün',
                  'DH Methoden & Tools': 'Dunkelgrün',
                  'DH Projekt': 'Beige',
                  'Programmieren für DH': 'Dunkelrot',
                  'Statistik für DH': 'Blau',
                  'Umgang mit dem PC': 'Orange',
                  'Schlüsselkompetenzen Zertifikat "Digital Humanities"': 'Grau',}


df['color'] = df['Bereich'].apply(lambda x: RUB_palette[bereich_colors[x]])

RUB_colors = list(RUB_palette.values())

In [ ]:
import plotly.io as pio
pio.templates

In [ ]:
df['Modul'] = df.apply(lambda x: x.Modul if x.Bereich != 'Umgang mit dem PC' else 'Methodenmodul', axis=1)


In [ ]:
head = '''
<head>
<meta charset="utf-8" />
<a href="https://forms.gle/GMZLgXxU9dNHsnPS9" target="_blank"><b>Feedback</b> zu eventuell <b>fehlenden Skills</b></a>
  <center>
    <h1 style="font-size: 25;"> Digital-Humanities-Kompetenzen </h1>
  </center>
</head>
'''

def add_header(html):
  with open(html, 'r') as f:
    content = f.read()
    content = re.sub(r'<head>.*?</head>',head,content)
  with open(html, 'w') as o:
    o.write(content)

# Ohne Module

In [ ]:
import plotly.express as px

fig = px.treemap(df, path=[
    # px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    # 'Modul',
    'Bereich',
    'Unterbereich',
    'AS'
],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # color=df['color'],
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    color='Bereich',
    color_discrete_sequence=df['color'],
    color_discrete_map=dict(zip(df['Bereich'], df['color']))
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    # title={
    # 'text': 'Schlüsselkompetenzen Zertifikat "Digital Humanities"',  # Set your headline text here
    # 'y':0.98,  # Position the title slightly below the top edge
    # 'x':0.5,   # Center-align the title
    # 'xanchor': 'center',
    # 'yanchor': 'top'
    # },
    # template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        color="Black"
    )
)
fig.show()
fig.write_html("AS_Bereiche.html",  include_plotlyjs='cdn')
add_header("AS_Bereiche.html")
fig.write_image(f"img/AS_Bereiche.svg")

## Nach Workshop-Priorisierung

In [ ]:
df.keys()

In [ ]:
import plotly.express as px

df['weight_hover'] = df.apply(lambda df: f"Abstimmungsergebnisse<br>B.A.: {df['Punkteanzahl BA']}<br>M.A.: {df['Punkteanzahl MA']}", axis=1)

fig = px.treemap(df, path=[
    # px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    # 'Modul',
    'Bereich',
    'Unterbereich',
    'AS'
],
    values='Punkte gewichtet',
    custom_data=['weight_hover'],
    # color=df['color'],
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    # color_discrete_map={key: RUB_palette[bereich_colors[key]] for key in bereich_colors.keys()}
    color='Bereich',
    color_discrete_sequence=df['color'],
    color_discrete_map=dict(zip(df['Bereich'], df['color']))
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{customdata[0]}<br><br>%{label}')
fig.update_layout(
    # title={
    # 'text': 'Schlüsselkompetenzen Zertifikat "Digital Humanities"',  # Set your headline text here
    # 'y':0.98,  # Position the title slightly below the top edge
    # 'x':0.5,   # Center-align the title
    # 'xanchor': 'center',
    # 'yanchor': 'top'
    # },
    # template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        color="Black"
    )
)
fig.show()
fig.write_html("AS_Bereiche_gewichtet.html",  include_plotlyjs='cdn')
add_header("AS_Bereiche_gewichtet.html")
fig.write_image(f"img/AS_Bereiche_gewichtet.svg")

# Mit Modulen & AS

In [ ]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul',
    'Bereich',
    'Unterbereich',
    'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        # color="RebeccaPurple"
    )
)
fig.show()
fig.write_html("AS_3Module.html",  include_plotlyjs='cdn')

# Mit Modulen ohne AS

In [ ]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul',
    'Bereich',
    'Unterbereich',
    # 'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        # color="RebeccaPurple"
    )
)
fig.show()
fig.write_html("UB_3Module.html",  include_plotlyjs='cdn')

# 20-CP Variante

## Mit AS

In [ ]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul_Option2',
    'Bereich',
    'Unterbereich',
    'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.update_layout(
    template='seaborn',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
        # color="RebeccaPurple"
    )
)
fig.show()
fig.write_html("AS_4Module.html",  include_plotlyjs='cdn')

## Ohne AS

In [ ]:
import plotly.express as px

fig = px.treemap(df, path=[
    px.Constant('Schlüsselkompetenzen Zertifikat "Digital Humanities"'),
    'Modul_Option2',
    'Bereich',
    'Unterbereich',
    # 'AS'
    ],
    values='Prio_',
    # hover_data=['Zeitaufwand'],
    # marker_colors = RUB_colors
    # color='Zertifikatsbereich'
)
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
fig.update_traces(hovertemplate='%{label}')
fig.show()
fig.write_html("UB_4Module.html",  include_plotlyjs='cdn')

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

pio.templates["draft"] = go.layout.Template(
    layout_annotations=[
        dict(
            name="draft watermark",
            text="DRAFT",
            textangle=-30,
            opacity=0.1,
            font=dict(color="black", size=100),
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.5,
            showarrow=False,
        )
    ]
)

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

pio.templates["rub"] = go.layout.Template(
    layout_annotations=[
        dict(
            name="draft watermark",
            text="DRAFT",
            textangle=-30,
            opacity=0.1,
            font=dict(color="black", size=100),
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.5,
            showarrow=False,
        )
    ]
)

# Poster Plots

In [ ]:
import plotly.express as px

def plot_bereich(df, bereich):
    df = df[df['Bereich'] == bereich]
    fig = px.treemap(df, path=[
        px.Constant(bereich),
        'Unterbereich',
        'AS'
        ],
        # values='Prio_', # Skalierung nicht nötig für Plakate
        color_discrete_sequence =list(df['color'])
    )
    fig.update_traces(marker=dict(cornerradius=5))
    fig.update_layout(margin=dict(t=25, l=25, r=25, b=25))
    fig.update_traces(hovertemplate='%{label}')
    fig.update_layout(
        # template='seaborn',
        font=dict(
            # family="Courier New, monospace",
            size=15,  # Set the font size here
            color='black'
        )
    )
    fig.show()
    fig.write_image(f"img/{bereich.replace(' ', "_")}.svg")
    
print(RUB_palette)

for b in df.Bereich.value_counts().keys():
    # print(b)
    plot_bereich(df,b)